# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [8]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
from pprint import pprint

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,grytviken,-54.2811,-36.5092,37.15,70,20,12.10,GS,1706033786
1,1,ponta delgada,37.7333,-25.6667,65.19,77,75,16.11,PT,1706033864
2,2,nuku'alofa,-21.1333,-175.2000,79.14,94,75,5.75,TO,1706033865
3,3,belchatow,51.3688,19.3567,39.97,91,98,14.58,PL,1706033865
4,4,tura,25.5198,90.2201,54.32,68,1,2.30,IN,1706033866


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
%%capture --no-display

# Configure the map plot
hover_columns = ['City', 'Humidity']
plot = city_data_df.hvplot.points('Lng', 'Lat', geo=True, tiles='OSM', size=city_data_df['Humidity'], 
                                  width=800, height=550, c='Humidity', cmap='spectral', alpha=0.5, hover_cols=hover_columns)

# Display the map
plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (Humidity,_size,City)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [21]:
# Narrow down cities that fit criteria and drop any results with null values
ideal_weather_df = city_data_df[(city_data_df['Max Temp'] < 70) & (city_data_df['Max Temp'] > 65) & (city_data_df['Humidity'] < 80)]

# Drop any rows with null values
ideal_weather_df.dropna(inplace=True)

# Display sample data
ideal_weather_df

C:\Users\K\AppData\Local\Temp\ipykernel_19192\357253380.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ideal_weather_df.dropna(inplace=True)


,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
1,1,ponta delgada,37.7333,-25.6667,65.19,77,75,16.11,PT,1706033864
56,56,east london,-33.0153,27.9116,68.94,68,75,6.91,ZA,1706033709
59,59,bredasdorp,-34.5322,20.0403,66.09,73,0,19.82,ZA,1706033888
63,63,coquimbo,-29.9533,-71.3436,67.73,72,20,13.80,CL,1706033890
73,73,lompoc,34.6391,-120.4579,65.70,59,0,10.36,US,1706033895
94,94,quellon,-43.1167,-73.6167,65.61,57,20,10.71,CL,1706033904
101,101,ushuaia,-54.8000,-68.3000,69.46,40,0,18.41,AR,1706033907
122,122,ballina,-28.8667,153.5667,66.78,71,93,4.76,AU,1706033917
153,153,port alfred,-33.5906,26.8910,67.17,79,93,3.00,ZA,1706033931
178,178,constitucion,-35.3333,-72.4167,67.98,64,58,8.46,CL,1706033941


### Step 3: Create a new DataFrame called `hotel_df`.

In [22]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_weather_df[['City', 'Country', 'Lat', 'Lng', 'Humidity']].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ''

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
1,ponta delgada,PT,37.7333,-25.6667,77,
56,east london,ZA,-33.0153,27.9116,68,
59,bredasdorp,ZA,-34.5322,20.0403,73,
63,coquimbo,CL,-29.9533,-71.3436,72,
73,lompoc,US,34.6391,-120.4579,59,
94,quellon,CL,-43.1167,-73.6167,57,
101,ushuaia,AR,-54.8000,-68.3000,40,
122,ballina,AU,-28.8667,153.5667,71,
153,port alfred,ZA,-33.5906,26.8910,79,
178,constitucion,CL,-35.3333,-72.4167,64,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [23]:
# Set parameters to search for a hotel
radius = 10000
params = {
    'categories': 'accommodation.hotel',
    'limit': 1,
    'apiKey': geoapify_key
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat = row['Lat']
    lng = row['Lng']
    
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f'circle:{lng},{lat},{radius}'
    params["bias"] = f'proximity:{lng},{lat}'
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places?"


    # Make and API request using the params dictionaty
    name_response = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_response.json()

    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
ponta delgada - nearest hotel: Hotel Talisman
east london - nearest hotel: No hotel found
bredasdorp - nearest hotel: Victoria Hotel
coquimbo - nearest hotel: Hotel Iberia
lompoc - nearest hotel: Embassy Suites by Hilton Lompoc Central Coast
quellon - nearest hotel: Aliwen
ushuaia - nearest hotel: Apart Hotel Aires del Beagle
ballina - nearest hotel: Ballina Beach Resort
port alfred - nearest hotel: No hotel found
constitucion - nearest hotel: Hostal Alameda
hermanus - nearest hotel: Aloe guest house
port elizabeth - nearest hotel: Waterford Hotel
teguise - nearest hotel: Finca Las Laderas
bogale - nearest hotel: No hotel found
bati - nearest hotel: No hotel found
bokkos - nearest hotel: No hotel found
taoudenni - nearest hotel: No hotel found
khon buri - nearest hotel: The tree
plettenberg bay - nearest hotel: Milkwood Manor
porbandar - nearest hotel: Toran Tourist Bungalow
kruisfontein - nearest hotel: No hotel found
tingambato - nearest hotel: No hotel found
ea

,City,Country,Lat,Lng,Humidity,Hotel Name
1,ponta delgada,PT,37.7333,-25.6667,77,Hotel Talisman
56,east london,ZA,-33.0153,27.9116,68,No hotel found
59,bredasdorp,ZA,-34.5322,20.0403,73,Victoria Hotel
63,coquimbo,CL,-29.9533,-71.3436,72,Hotel Iberia
73,lompoc,US,34.6391,-120.4579,59,Embassy Suites by Hilton Lompoc Central Coast
94,quellon,CL,-43.1167,-73.6167,57,Aliwen
101,ushuaia,AR,-54.8000,-68.3000,40,Apart Hotel Aires del Beagle
122,ballina,AU,-28.8667,153.5667,71,Ballina Beach Resort
153,port alfred,ZA,-33.5906,26.8910,79,No hotel found
178,constitucion,CL,-35.3333,-72.4167,64,Hostal Alameda


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [24]:
%%capture --no-display

# Configure the map plot
hotel_columns = ['City', 'Humidity', 'Hotel Name', 'Country']
hotel_plot = plot = hotel_df.hvplot.points('Lng', 'Lat', geo=True, tiles='OSM', size=city_data_df['Humidity'], 
                                  width=800, height=550, c='Humidity', cmap='spectral', alpha=0.5, hover_cols=hotel_columns)

# Display the map
hotel_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (Humidity,_size,City,Hotel Name,Country)